In [5]:
!pip install python-terrier
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Need to install additional terrier package for PRF. It will take around 1 min
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

fatal: destination path 'terrier-prf' already exists and is not an empty directory.


E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
[Errno 2] No such file or directory: '/content/terrier-prf/'
/home/mark/Desktop/Academic/IR/Project


/home/mark/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


[INFO] Scanning for projects...
[INFO] ------------------------------------------------------------------------
[INFO] BUILD FAILURE
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  0.080 s
[INFO] Finished at: 2024-05-10T19:36:25+03:00
[INFO] ------------------------------------------------------------------------
[ERROR] The goal you specified requires a project to execute but there is no POM in this directory (/home/mark/Desktop/Academic/IR/Project). Please verify you invoked Maven from the correct directory. -> [Help 1]
[ERROR] 
[ERROR] To see the full stack trace of the errors, re-run Maven with the -e switch.
[ERROR] Re-run Maven using the -X switch to enable full debug logging.
[ERROR] 
[ERROR] For more information about the errors and possible solutions, please read the following articles:
[ERROR] [Help 1] http://cwiki.apache.org/confluence/display/MAVEN/MissingProjectException
/home/mark/Desktop/Academic/IR/Project
/home/mark/D

/home/mark/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1XxurvfReGGDoTue5IMiETBKGj6mO_ZRD'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentFile("Data.zip")))

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd

import os
import re

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


# Preprocessing

In [ ]:
!unzip Data.zip -d Data

Archive:  Data.zip
  inflating: Data/01. How Your Nervous System Works and Changes.md  
  inflating: Data/02. Master Your Sleep and Be More Alert When Awake.md  
  inflating: Data/03. Using Science to Optimize Sleep Learning and Metabolism.md  
  inflating: Data/04. Find Your Temperature Minimum to Defeat Jetlag Shift Work and Sleeplessness.md  
  inflating: Data/05. Understanding and Using Dreams to Learn and to Forget.md  
  inflating: Data/06. How to Focus to Change Your Brain.md  
  inflating: Data/07. Using Failures Movement and Balance to Learn Faster.md  
  inflating: Data/08. Optimize Your Learning and Creativity with Science-based Tools.md  
  inflating: Data/09. Control Pain and Heal Faster with Your Brain.md  
  inflating: Data/10. Tools for Managing Stress and Anxiety.md  
  inflating: Data/11. How Foods and Nutrients Control Our Moods.md  
  inflating: Data/12. How to Increase Motivation and Drive.md  
  inflating: Data/13. The Science of Emotions and Relationships.md  
  

In [4]:
data = []
dir = os.listdir("Data")
for f in dir:
    try:
        file = open("./Data/" + f,encoding="utf-8" )    
        fContent = file.read()
        fContent = re.sub(r"\[.*?\]\(http.*?\)", "", fContent)
        fContent = re.sub(r"(\*\*|__)(.*?)(\*\*|__)","",fContent)
        fContent = re.sub(r"<.*?>", "", fContent)
        # fContent = re.sub(r"\(\d{2}:\d{2}:\d{2}\)", "", fContent)
        fContent = re.sub(r"\n\s*\n", "\n", fContent)
        fContent = fContent.replace("> Written with .","")
        fileDic = {}
        fileDic["name"] = f
        fileDic["raw_data"] = fContent
        data.append(fileDic)
    finally:
        continue


FileNotFoundError: [Errno 2] No such file or directory: 'Data'

In [ ]:
df = pd.DataFrame(data)
df["raw_data"] = df["raw_data"].astype(str)
df.info()

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
print(stopwords.words('english'))

In [ ]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stopwords.words('english')] #Lower is used to normalize al the words make them in lower case
    return ' '.join(filtered_tokens)
df["proc_data"] = df["raw_data"].apply(remove_stopwords)


In [ ]:
stemmer = PorterStemmer()
def Steem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)
df['proc_data']=df['proc_data'].apply(Steem_text)
df.head()

In [ ]:
def preproccess(text):
  t = remove_stopwords(text)
  t = Steem_text(t)
  return t

In [ ]:
df['docno'] = range(1, len(df) + 1)
df['docno'] = df['docno'].astype(str)
df['docno'] = "d" + df['docno']
df.head()

# Indexing

In [ ]:
indexer = pt.DFIndexer("./myFirstIndex")

# index the text, record the docnos as metadata
index_ref = indexer.index(df["proc_data"], df["docno"])
index = pt.IndexFactory.of(index_ref)


# Query Processing

In [ ]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)

In [ ]:
def queryIndex(query):
    query = preproccess(query)
    #we will call the search function using our retrieval model we set up above
    results=tfidf_retr.search(query)
    results
    return pd.merge(df[df["docno"].isin(list(results['docno']))],results).sort_values("rank")[["name","raw_data","rank"]]
    

In [ ]:
queryIndex("learning during dreaming and unlearning during dreaming")

In [ ]:
from flask import Flask, request, render_template
 
# Flask constructor takes the name of 
# current module (__name__) as argument.
app = Flask(__name__)
 
# The route() function of the Flask class is a decorator, 
# which tells the application which URL should call 
# the associated function.
@app.route('/')
# ‘/’ URL is bound with hello_world() function.
def hello_world():
    return render_template('index.html')

@app.route('/view/<string:q>', methods=['GET'])
def view(q):
    file = open("./Data/" + q,encoding="utf-8" )
    fContent = file.read()
    return render_template('view.html',content=fContent)
# ‘/’ URL is bound with hello_world() function.
def hello_world():
    return render_template('index.html')

@app.route('/search/<string:q>', methods=['GET'])
def json_search(q):
    results = queryIndex(q)
    return results[["name","rank"]].to_json(orient="records")
 
# main driver function
if __name__ == '__main__':
 
    # run() method of Flask class runs the application 
    # on the local development server.
    app.run()